The DAYMET dataset is _______. DAYMET data is available through Oak Ridge National Laboratory Distributed Active Archive Center (ORNL DAAC), which houses and distributes a large variety of NASA data. (Read more [here](https://daac.ornl.gov/about/).)

When attempting to query these datasets programmatically, it helps to find the source DOI in the ORNL DAAC catalog. We are looking for DAYMET daily data for continental North America, which we find at this DOI: ```https://doi.org/10.3334/ORNLDAAC/2129```. Looking at the the DOI URL suffix, we can see that the dataset identifier in the ORNL DAAC catalog is ```2129```. This identifier is important, as it will become part of the URL we will build to query the ORNL DAAC THREDDS server.

While there are several data access services available on the ORNL DAAC THREDDS server, we are going to use the OPeNDAP access option to get our data. The URL structure for all OPeNDAP requests is as follows:

-  ```https://thredds/dodsC/ornldaac/<dataset_id>/<filename>```

And the filename structure is as follows:

- ```daymet_v4_daily_<region>_<variable>_<year>.nc```

_(We found this by browsing through the ORNL-DAAC catalog until we got to an individual item's page, and noting it's OPeNDAP access URL pattern. See [this page](https://thredds.daac.ornl.gov/thredds/catalog/ornldaac/2129/catalog.html?dataset=2129/daymet_v4_daily_na_prcp_1996.nc) for an example item, and check out the OPeNDAP URL structure under "Access", item #1.)_



Choose the DAYMET region of interest. The choices are:

- na: continental North America
- hi: Hawaii
- pr: Puerto Rico

Choose the DAYMET variables that we want to download. The choices are:

- all: all variables below
- dayl: duration of daylight
- prcp: daily total precipitation
- srad: incident shortwave radiation flux density (average over daylight period)
- swe: snow water equivalent
- tmax: maximum 2-meter air temperature
- tmin: minimum 2-meter air temperature
- vp: water vapor pressure (daily average)
(listed here: https://daymet.ornl.gov/single-pixel-tool-guide#:~:text=Variables%20and%20Units%3A)

In [9]:
from pydap.client import open_url
import xarray as xr

In [13]:
dataset_id = '2129'
region = 'na'
var = 'prcp'
start_year, end_year = 2000, 2003
years = [str(y) for y in range(start_year, end_year+1)]


urls = []
for y in years:
    filename = 'daymet_v4_daily_' + region + '_' + var + '_' + y + '.nc'
    url = 'https://thredds.daac.ornl.gov/thredds/dodsC/ornldaac/' + dataset_id + '/' + filename
    urls.append(url)

print(urls)

['https://thredds.daac.ornl.gov/thredds/dodsC/ornldaac/2129/daymet_v4_daily_na_prcp_2000.nc', 'https://thredds.daac.ornl.gov/thredds/dodsC/ornldaac/2129/daymet_v4_daily_na_prcp_2001.nc', 'https://thredds.daac.ornl.gov/thredds/dodsC/ornldaac/2129/daymet_v4_daily_na_prcp_2002.nc', 'https://thredds.daac.ornl.gov/thredds/dodsC/ornldaac/2129/daymet_v4_daily_na_prcp_2003.nc']


In [14]:
datasets = []

for url in urls:
    daymet_opendap = open_url(url) 
    
    # Xarray DataSet - opening dataset via remote OPeNDAP 
    ds = xr.open_dataset(xr.backends.PydapDataStore(daymet_opendap), decode_coords="all")
    datasets.append(ds)
        

In [16]:
datasets[0]

<xarray.Dataset>
Dimensions:                  (x: 7814, y: 8075, time: 365, nv: 2)
Coordinates:
  * x                        (x) float32 -4.56e+06 -4.559e+06 ... 3.253e+06
  * y                        (y) float32 4.984e+06 4.983e+06 ... -3.09e+06
    lat                      (y, x) float32 ...
    lon                      (y, x) float32 ...
  * time                     (time) datetime64[ns] 2000-01-01T12:00:00 ... 20...
    time_bnds                (time, nv) datetime64[ns] ...
    lambert_conformal_conic  int16 ...
Dimensions without coordinates: nv
Data variables:
    yearday                  (time) int16 ...
    prcp                     (time, y, x) float32 ...
Attributes:
    start_year:                      2000
    source:                          Daymet Software Version 4.0
    Version_software:                Daymet Software Version 4.0
    Version_data:                    Daymet Data Version 4.0
    Conventions:                     CF-1.6
    citation:                        Please see http://daymet.ornl.gov/ for c...
    references:                      Please see http://daymet.ornl.gov/ for c...
    DODS_EXTRA.Unlimited_Dimension:  time